In [1]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "mnist"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:5000"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

#from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

In [151]:
from __future__ import annotations

import os
from pathlib import Path
from posixpath import join as urljoin
from typing import Any
from urllib.parse import urlparse, urlunparse
import inspect
import requests
from typing import Any, cast

DEBUG = True
class Session(object):
    pass

def create_data_dict(**kwargs):
    return kwargs

def debug(url, data=None):
    if DEBUG:
        print("url:", url)
        if (data):
            print("data:", data)
def get(session, endpoint, *features):
    debug(urljoin(endpoint, *features))
    return make_request(session, 'get', endpoint, None, *features)
def post(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    return make_request(session, 'post', endpoint, data, *features)
def delete(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    return make_request(session, 'delete', endpoint, data, *features)
def put(session, endpoint, data, *features):
    debug(urljoin(endpoint, *features), data)
    return make_request(session, 'put', endpoint, data, *features)


def make_request(session, method, endpoint, data, *features):
    url = urljoin(endpoint, *features)
    method = getattr(session, method)
    try: 
        if (data):
            ret = method(url, json=data)
        else:
            ret = method(url)
    except* requests.ConnectionError as e:
        handle_error(e) # this may occur if the connection was unsuccessful
    
    try:
        ret = ret.json()
    except* requests.JSONDecodeError as e:
        handle_error(ret.text) # this occurs if the URL does not exist
    return ret
def handle_error(error):
    print(error)

class DioptraClient(object):
    def __init__(self, session, address=None, api_version="v1") -> None:
        self._session = session
        self._users = UsersClient(session, "users", address, api_version)
        self._auth = AuthClient(session, "auth", address, api_version)
    @property
    def users(self):
        return self.get_endpoint(self._users)
    @property
    def auth(self):
        return self.get_endpoint(self._auth)

    def get_endpoint(self, ep):
        ep.session = self._session
        return ep

class Endpoint(object):
    def __init__(self, session, ep_name, address, api_version) -> None:
        address = (
            f"{address}/api/{api_version}"
            if address
            else f"{os.environ['DIOPTRA_RESTAPI_URI']}/api/{api_version}"
        )
        self._scheme, self._netloc, self._path, _, _, _ = urlparse(address)
        self._ep_name = ep_name
        self._session = session
    @property
    def session(self):
        return self._session
    @session.setter
    def session(self, s):
        self._session = s
    @property
    def url(self):
        return self.def_endpoint(self._ep_name)
    def def_endpoint(self, name):
        '''creates base url for an endpoint by name'''
        return urlunparse(
            (self._scheme, self._netloc, urljoin(self._path, name + "/"), "", "", "")
        )

class UsersClient(Endpoint):
    def get_all(self):
        '''gets all users'''
        return get(self.session, self.url)
    def create(self, username, email, password, confirm_password):
        '''creates a user'''
        d = {"username":username,
             "email": email,
             "password": password,
             "confirmPassword": confirm_password}
        return post(self.session, self.url, d)
    def get_by_id(self, id):
        '''get a user by id'''
        return get(self.session, self.url, str(id))
    def update_password_by_id(self, id, old_password, new_password, confirm_new_password):
        '''change a user's password by id'''
        d = {"oldPassword":old_password,
             "newPassword": new_password,
             "confirmNewPassword": confirm_new_password}
        return post(self.session, self.url, d, str(id), 'password')
    def current(self):
        '''get the current user'''
        return get(self.session, self.url, 'current')
    def delete_current(self, password):
        '''delete the current user'''
        d = {"password":password}
        return delete(self.session, self.url, d, 'current')
    def modify_current(self, username, email):
        '''modify the current user'''
        d = {"username":username,
             "email": email}
        return put(self.session, self.url, d, 'current')
    def modify_current_password(self, old_password, new_password, confirm_new_password):
        '''modify the current user's password'''
        d = {"oldPassword":old_password,
             "newPassword": new_password,
             "confirmNewPassword": confirm_new_password}
        return post(self.session, self.url, d, 'current', 'password')
    def failed_user_post(self):
        return post(self.session, self.url, {'a':'doesnotexist'})
    def failed_user_get(self):
        return get(self.session, self.url, 'doesnotexist')
class AuthClient(Endpoint):
    def login(self, username, password):
        d = {"username":username,
             "password": password}
        return post(self.session, self.url, d, 'login')
    def logout(self, everywhere):
        d = {"everywhere": everywhere}
        return post(self.session, self.url, d, 'logout')

In [152]:
client = DioptraClient(requests.Session())

In [153]:
client.users.create('testuser','testuser@gmail.com','testuserpassword','testuserpassword')

url: http://localhost:5000/api/v1/users/
data: {'username': 'testuser', 'email': 'testuser@gmail.com', 'password': 'testuserpassword', 'confirmPassword': 'testuserpassword'}


{'username': 'testuser',
 'email': 'testuser@gmail.com',
 'id': 6,
 'groups': [{'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}],
 'createdOn': '2024-06-19T18:05:09.732129+00:00',
 'lastModifiedOn': '2024-06-19T18:05:09.732129+00:00',
 'lastLoginOn': None,
 'passwordExpiresOn': '2025-06-19T18:05:09.732129+00:00'}

In [154]:
client.users.create('testuser2','testuser2@gmail.com','testuserpassword','testuserpassword')

url: http://localhost:5000/api/v1/users/
data: {'username': 'testuser2', 'email': 'testuser2@gmail.com', 'password': 'testuserpassword', 'confirmPassword': 'testuserpassword'}


{'message': 'Bad Request - The username on the registration form is not available. Please select another and resubmit.'}

In [155]:
client.auth.login('testuser','testuserpassword')

url: http://localhost:5000/api/v1/auth/login
data: {'username': 'testuser', 'password': 'testuserpassword'}


{'username': 'testuser', 'status': 'Login successful'}

In [156]:
client.users.get_all()

url: http://localhost:5000/api/v1/users/


{'index': 0,
 'isComplete': True,
 'totalNumResults': 5,
 'first': '/api/v1/users?index=0&pageLength=10',
 'data': [{'username': 'string', 'email': 'string', 'id': 1},
  {'username': 'aa', 'email': 'aa', 'id': 2},
  {'username': 'newuser', 'email': 'newemail@email.com', 'id': 3},
  {'username': 'testuser2', 'email': 'testuser2@gmail.com', 'id': 4},
  {'username': 'testuser', 'email': 'testuser@gmail.com', 'id': 6}]}

In [157]:
client.users.get_by_id('1')

url: http://localhost:5000/api/v1/users/1


{'username': 'string', 'email': 'string', 'id': 1}

In [158]:
client.users.get_by_id('2')

url: http://localhost:5000/api/v1/users/2


{'username': 'aa', 'email': 'aa', 'id': 2}

In [159]:
client.users.current()

url: http://localhost:5000/api/v1/users/current


{'username': 'testuser',
 'email': 'testuser@gmail.com',
 'id': 6,
 'groups': [{'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}],
 'createdOn': '2024-06-19T18:05:09.732129+00:00',
 'lastModifiedOn': '2024-06-19T18:05:09.732129+00:00',
 'lastLoginOn': '2024-06-19T18:05:10.212491+00:00',
 'passwordExpiresOn': '2025-06-19T18:05:09.732129+00:00'}

In [160]:
client.users.update_password_by_id(client.users.current()['id'],'testuserpassword','newtestuserpassword','newtestuserpassword')

url: http://localhost:5000/api/v1/users/current
url: http://localhost:5000/api/v1/users/6/password
data: {'oldPassword': 'testuserpassword', 'newPassword': 'newtestuserpassword', 'confirmNewPassword': 'newtestuserpassword'}


{'status': 'Password Change Success'}

In [161]:
client.users.modify_current('testuser','newemail@email.com')

url: http://localhost:5000/api/v1/users/current
data: {'username': 'testuser', 'email': 'newemail@email.com'}


{'message': "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}

In [162]:
client.auth.login('testuser','newtestuserpassword')

url: http://localhost:5000/api/v1/auth/login
data: {'username': 'testuser', 'password': 'newtestuserpassword'}


{'username': 'testuser', 'status': 'Login successful'}

In [163]:
client.users.modify_current('testuser','newemail@email.com')

url: http://localhost:5000/api/v1/users/current
data: {'username': 'testuser', 'email': 'newemail@email.com'}


{'username': 'testuser',
 'email': 'newemail@email.com',
 'id': 6,
 'groups': [{'id': 1, 'name': 'public', 'url': '/api/v1/groups/1'}],
 'createdOn': '2024-06-19T18:05:09.732129+00:00',
 'lastModifiedOn': '2024-06-19T18:05:11.913576+00:00',
 'lastLoginOn': '2024-06-19T18:05:11.709840+00:00',
 'passwordExpiresOn': '2025-06-19T18:05:11.402927+00:00'}

In [164]:
client.users.modify_current_password('newtestuserpassword','newnewtestuserpassword','newnewtestuserpassword')

url: http://localhost:5000/api/v1/users/current/password
data: {'oldPassword': 'newtestuserpassword', 'newPassword': 'newnewtestuserpassword', 'confirmNewPassword': 'newnewtestuserpassword'}


{'status': 'Password Change Success'}

In [165]:
client.auth.logout(True)

url: http://localhost:5000/api/v1/auth/logout
data: {'everywhere': True}


{'message': "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}

In [166]:
client.auth.login('testuser','newnewtestuserpassword')

url: http://localhost:5000/api/v1/auth/login
data: {'username': 'testuser', 'password': 'newnewtestuserpassword'}


{'username': 'testuser', 'status': 'Login successful'}

In [167]:
client.users.delete_current('newnewtestuserpassword')

url: http://localhost:5000/api/v1/users/current
data: {'password': 'newnewtestuserpassword'}


{'status': 'Success'}

In [168]:
client.users.failed_user_post() # if we get the schema wrong, it does return JSON

url: http://localhost:5000/api/v1/users/
data: {'a': 'doesnotexist'}


{'schema_errors': {'a': ['Unknown field.']}}

In [169]:
client.users.failed_user_get() # if we get the URL wrong, it does not return JSON

url: http://localhost:5000/api/v1/users/doesnotexist
<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>



<Response [404]>